# registering_your_model_with_the_benchmark

A step-by-step tutorial for how to register a new model with the benchmark.

This tutorial assumes you have followed the [installation instructions](https://nrel.github.io/BuildingsBench/#installation) for BuildingsBench.

In [1]:
from buildings_bench.models.base_model import BaseModel
import torch
import torch.nn as nn

%load_ext autoreload
%autoreload 2

## 1. Model Implementation

Let's create an example PyTorch model that is an MLP performing direct regression on the 24 hour forecast.

In [3]:
class MyModel(BaseModel):

    def __init__(self, 
                 hidden_size,
                 context_len=168,
                 pred_len=24,
                 continuous_loads=True):
        """Init method for MyModel.
        
        Args:
            hidden_size (int): size of hidden layer
            context_len (int): length of context window
            pred_len (int): length of prediction window
            continuous_loads (bool): whether this model uses continuous load values
        """
        super().__init__(context_len, pred_len, continuous_loads)

        # Our model will be a simple MLP with two hidden layers
        # and will output two values (mean and std dev) for each hour.
        self.mlp = nn.Sequential(
            nn.Linear(context_len, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, pred_len*2)
        )

    def forward(self, x):
        """
        `x` is a dictionary with the following keys:

        ```
        'load': torch.Tensor,               # (batch_size, seq_len, 1)
        'building_type': torch.LongTensor,  # (batch_size, seq_len, 1)
        'day_of_year': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'hour_of_day': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'day_of_week': torch.FloatTensor,   # (batch_size, seq_len, 1)
        'latitude': torch.FloatTensor,      # (batch_size, seq_len, 1)
        'longitude': torch.FloatTensor,     # (batch_size, seq_len, 1)
        ```

        This model only uses the 'load'.
        """
        # (batch_size, self.context_len)
        x = x['load'][:, :self.context_len, 0]
        out = self.mlp(x)  # (batch_size, self.pred_len*2)
        return out.view(-1, self.pred_len, 2) # (batch_size, self.pred_len, 2)
    
    def loss(self, x, y):
        """
        Args:
            x (torch.Tensor): preds of shape (batch_size, seq_len, 2)
            y (torch.Tensor): targets of shape (batch_size, seq_len, 1)
        Returns:
            loss (torch.Tensor): scalar loss
        """
        return nn.functional.gaussian_nll_loss(x[:, :, 0].unsqueeze(2), y,
                                       nn.functional.softplus(x[:, :, 1].unsqueeze(2))**2)
    
    def predict(self, x):
        """
        Args:
            x (Dict): dictionary of input tensors
        Returns:
            predictions (torch.Tensor): of shape (batch_size, pred_len, 1)
            distribution_parameters (torch.Tensor): of shape (batch_size, pred_len, -1)
        """
        out = self.forward(x)
        means = out[:, :, 0].unsqueeze(2)
        stds = nn.functional.softplus(out[:, :, 1].unsqueeze(2))
        return means, torch.cat([means, stds], dim=2)
    
    def unfreeze_and_get_parameters_for_finetuning(self):
        """For transfer learning."""
        return self.parameters()
    
    def load_from_checkpoint(self, checkpoint_path):
        """Describe how to load model from a checkpoint."""
        self.load_state_dict(torch.load(checkpoint_path)['model'])

## 2. Adding your model to the model registry

Store this class definition in a file called my_model.py under `./buildings_bench/models/my_model.py`.
At the top of `./buildings_bench/models/__init__.py`, add the following import statement:

```python
from buildings_bench.models.my_model import MyModel
```

Then, add your model to the `model_registry` dictionary:

```python
model_registry = {
    ...
    'MyModel': MyModel,
    ...
}
```

## 3. Creating a TOML config file

Create a TOML config file under `./configs/MyModel.toml` with each keyword argument your model expects in its constructor (i.e., the hyperparameters for your model) and any argparse args you wish to override for the script you want to run.

The example model's config file `MyModel.toml` should look something like this:

```toml
[model]
hidden_size = 1024
context_len = 168
pred_len = 24
continuous_loads = true

[pretrain]
batch_size = 256
init_scale = 0.02
warmup_steps = 10000
lr = 0.00006
scheduler_steps = 162760
apply_scaler_transform = 'boxcox'

[transfer_learning]
apply_scaler_transform = 'boxcox'
```

We specified the model's hidden size, context length, prediction length, and whether the model should be trained on continuous loads or not. 

For pretraining, we specified the batch size, learning rate, and number of warmup steps. We also specified the initial scale for the weight initialization and the number of steps for the learning rate scheduler. Finally, we specified to apply a Box-Cox transform to the loads.

For transfer learning, we will use the default values for the batch size and learning rate, but we specify we want to apply the same Box-Cox transform to the loads.

## 4. Running the scripts

### Pretraining

`python3 scripts/pretrain.py --model MyModel`

This script is implemented with PyTorch `DistributedDataParallel`, so it can be launched with `torchrun`. See `./scripts/pretrain.sh` for an example.

### Zero-shot STLF

`python3 scripts/zero_shot.py --model MyModel --checkpoint /path/to/checkpoint.pt`

### Transfer Learning for STLF

`python3 scripts/transfer_learning_torch.py --model MyModel --checkpoint /path/to/checkpoint.pt`